In [ ]:
!pip install langchain 
!pip install openai 
!pip install python-environ 
!pip install psycopg2 

In [ ]:
import psycopg2

In [ ]:
username = 'wsuser'
password = 'Solliance123'
host = 'pgsqldevilkdittaflex16.postgres.database.azure.com'
port = '5432'
dbname = 'ailabs'

connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host=host,
    port=5432,
    user=username,
    password=password,
    database=dbname
)

In [ ]:
#load the csv data to the database
import pandas as pd
from sqlalchemy import create_engine

db_engine = create_engine(connection_string)

In [ ]:
df = pd.read_csv('./data/calendar.csv')
df.to_sql('calendar', db_engine, if_exists='replace', index=False)

df = pd.read_csv('./data/listings.csv')
df.to_sql('listings', db_engine, if_exists='replace', index=False)

df = pd.read_csv('./data/reviews.csv')
df.to_sql('reviews', db_engine, if_exists='replace', index=False)

In [ ]:
from langchain.chat_models import AzureChatOpenAI

azure_endpoint = "https://fllm4693d-openai.openai.azure.com/"
azure_key = '1b13f0a8efa1432796a3dab8f0e62a2d'

llm=AzureChatOpenAI(
    temperature=0,
    openai_api_key =azure_key,
    openai_api_base = azure_endpoint,
    openai_api_version="2023-03-15-preview",
    openai_api_type="azure",
    model_name="gpt4",
    deployment_name="completions4"
)

In [ ]:
def handle_parsing_errors():
    pass

In [ ]:
from langchain_experimental.sql.base import SQLDatabaseChain
#from langchain_experimental.sql.base.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.callbacks import get_openai_callback

from langchain import SQLDatabase

db = SQLDatabase.from_uri(connection_string)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    handle_parsing_errors=handle_parsing_errors
)

# Create db chain
QUERY = """
Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.
Use the following format:

"Question": "Question here"
"SQLQuery": "SQL Query to run"
"SQLResult": "Result of the SQLQuery"
"Answer": "Final answer here"

"{question}"
"""

question_text = "What are the details of the rental that is closest to the Space Needle?"

question = QUERY.format(question=question_text)

with get_openai_callback() as cb:
    res = agent_executor.run(question)

In [35]:
print(res)

"Question": "Which rental is closest to the Space Needle and what is its address?"
"SQLQuery": "SELECT id, street, latitude, longitude FROM listings ORDER BY ((latitude-47.6205)*(latitude-47.6205) + (longitude+122.3493)*(longitude+122.3493)) ASC LIMIT 10;"
"SQLResult": "[(6613601, '5th Avenue North, Seattle, WA 98109, United States', 47.61907155891863, -122.34779881165326), ...]"
"Answer": "The closest rental to the Space Needle is located at 5th Avenue North, Seattle, WA 98109, United States."
